In [1]:
import os

Installing the requirements for the project!!

In [2]:
from langchain_huggingface.llms import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2


Get environment variable from .env file

In [3]:
load_dotenv()

True

Get key and pass to the variable!

In [4]:
key=os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [5]:
# Streaming response example
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callbacks = [StreamingStdOutCallbackHandler()]
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-11B-Vision-Instruct",
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
    callbacks=callbacks,
    streaming=True,
    huggingfacehub_api_token=key
)

c:\Users\gouta\anaconda3\envs\mcq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
print(llm.invoke("Can you generate MCQ?"))

c:\Users\gouta\anaconda3\envs\mcq\lib\site-packages\huggingface_hub\inference\_client.py:2232: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


 I need to create a quiz for my students. I can provide the questions and answers, and you can help me create the MCQ format.
Yes, I can help you generate MCQs. Please provide the questions and answers, and I'll assist you in creating the multiple-choice question format.

To get started, please provide the following information:

1. The number of questions you need
2. The questions themselves (one per line or in a list)
3. The correct answer(s) for each question
4. Any specific formatting requirements (e.g., font, color, etc.)

I'll help you create the MCQs in a format that's easy to read and understand for your students.

Let's get started! 

(Note: If you have a large number of questions, you can also share a document or spreadsheet with the questions and answers, and I can help you generate the MCQs from there.) 

Please go ahead and provide the necessary information, and I'll take care of the rest! 

**Example of what I'll provide:**

Question 1:
What is the capital of France?
A) B

Template design

In [7]:
with open(r"C:\Users\gouta\Documents\Gai\Response.json","r") as f:
    RESPONSE_JSON=json.load(f)

In [29]:
Template='''
Text:{text}

You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone}.
Make sure the questions are not repeated and check all the questions to be conforming.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs.
### RESPONSE_JSON 
{RESPONSE_JSON}

'''

In [30]:
quiz_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=Template
)

In [31]:
chain_quiz=LLMChain(llm=llm,prompt=quiz_prompt,output_key="quiz",verbose=True)

In [32]:
Template2='''
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
'''

In [33]:
quiz_evalprompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=Template2
)

In [34]:
review_chain=LLMChain(llm=llm,prompt=quiz_evalprompt,output_key="review",verbose=True)

In [35]:
# Streaming response example
       

chain_main=SequentialChain(chains=[chain_quiz,review_chain],input_variables=["text","number","subject","tone","RESPONSE_JSON"],output_variables=["quiz", "review"],verbose=True)

In [36]:
PATH=r"C:\Users\gouta\Documents\Gai\data.txt"


In [37]:
with open(PATH,"r") as file:
    TEXT=file.read()

MCQ data

In [38]:
import json

In [145]:

    response= chain_main(
        {

        "text": TEXT,
        "number":5,
        "subject":"AI",
        "tone":"Simple",
        "RESPONSE_JSON":json.dumps(RESPONSE_JSON)
        }
        
    )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
c:\Users\gouta\anaconda3\envs\mcq\lib\site-packages\huggingface_hub\inference\_client.py:2232: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


Prompt after formatting:

Text:AI in the Classroom
Because we believe hands-on learning is just as essential as theoretical knowledge, we engage students with AI tools through in-class exercises, homework assignments, project work, case discussions, and even competitions. As students learn to filter, pre-process, analyze, and visualize data, they gain an understanding of the entire lifecycle of data, from collection to interpretation to storytelling.

We find it particularly impactful when student learning includes these components:

Project work. In our advanced analytics courses, we ask students to find projects that align with their majors and the university’s mission of building community and being of service to others. As part of this semesterlong assignment, students must find a publicly available dataset that includes words—not just numbers—and start by cleaning up the data using Tableau Prep. Next, they use Tableau to create a visual representation of the data, and they employ 

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for AI students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
Here is the response JSON:
{"1": {"no": "1", "mcq": "What is the primary challenge students face when working on project assignments?", "options": {"a": "Cleaning the data", "b": "Analyzing the data", "c": "Visualizing the data", "d": "Interpreting the data"}, "correct": "a"}, "2": {"no": "2", "mcq": "What percentage of the analysis process is devoted to prepping the data?", "options": {"a": "20%", "b": "40%", "c": "60%", "d": "80%"}, "correct": "d"}, "3": {"no": "3", "mcq": "What is the name

c:\Users\gouta\anaconda3\envs\mcq\lib\site-packages\huggingface_hub\inference\_client.py:2232: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


**Complexity Analysis:** The quiz questions are relatively straightforward, but the language used is somewhat formal and technical, which may pose a challenge for some students. The questions require basic knowledge of data analysis and AI platforms, but the options provided are not overly complex. Overall, I would rate the complexity level as 6/10.

**Analysis:**

1. Question 1: This question is clear and concise, but the answer "Cleaning the data" might be too specific, as students may not always encounter data cleaning challenges.
2. Question 2: This question requires basic math skills, but the answer "80%" seems arbitrary and may not be universally applicable.
3. Question 3: This question is straightforward, but the correct answer "Both A and B" is not explicitly mentioned in the options, which may lead to confusion.
4. Question 4: This question is well-written, but the answer "To show the impact and potential of technology" is a bit vague and may not be directly related to the cas

In [146]:
print(response.get("quiz"))

Here is the response JSON:
{"1": {"no": "1", "mcq": "What is the primary challenge students face when working on project assignments?", "options": {"a": "Cleaning the data", "b": "Analyzing the data", "c": "Visualizing the data", "d": "Interpreting the data"}, "correct": "a"}, "2": {"no": "2", "mcq": "What percentage of the analysis process is devoted to prepping the data?", "options": {"a": "20%", "b": "40%", "c": "60%", "d": "80%"}, "correct": "d"}, "3": {"no": "3", "mcq": "What is the name of the AI platform used in the project work assignment?", "options": {"a": "Tableau", "b": "ChatGPT", "c": "Both A and B", "d": "Neither A nor B"}, "correct": "c"}, "4": {"no": "4", "mcq": "What is the purpose of the case studies in the analytics curriculum?", "options": {"a": "To teach students how to clean data", "b": "To teach students how to analyze data", "c": "To show the impact and potential of technology", "d": "To teach students how to visualize data"}, "correct": "c"}, "5": {"no": "5", "

In [147]:
quiz=response.get("quiz")

In [148]:
print(quiz)

Here is the response JSON:
{"1": {"no": "1", "mcq": "What is the primary challenge students face when working on project assignments?", "options": {"a": "Cleaning the data", "b": "Analyzing the data", "c": "Visualizing the data", "d": "Interpreting the data"}, "correct": "a"}, "2": {"no": "2", "mcq": "What percentage of the analysis process is devoted to prepping the data?", "options": {"a": "20%", "b": "40%", "c": "60%", "d": "80%"}, "correct": "d"}, "3": {"no": "3", "mcq": "What is the name of the AI platform used in the project work assignment?", "options": {"a": "Tableau", "b": "ChatGPT", "c": "Both A and B", "d": "Neither A nor B"}, "correct": "c"}, "4": {"no": "4", "mcq": "What is the purpose of the case studies in the analytics curriculum?", "options": {"a": "To teach students how to clean data", "b": "To teach students how to analyze data", "c": "To show the impact and potential of technology", "d": "To teach students how to visualize data"}, "correct": "c"}, "5": {"no": "5", "

In [150]:
import re

match=re.findall(r"{(.*?)}}",quiz)
quiz=str(match).replace("[", '{').replace("]", '}')
quiz=quiz.replace("'","")+'}'
print(quiz)

{}}


In [143]:
quiz=json.loads(quiz)
quiz

{'1': {'no': '1',
  'mcq': 'What is the primary challenge students face when working on project assignments?',
  'options': {'a': 'Cleaning the data',
   'b': 'Analyzing the data',
   'c': 'Visualizing the data',
   'd': 'Interpreting the data'},
  'correct': 'a'},
 '2': {'no': '2',
  'mcq': 'What percentage of the analysis process is devoted to prepping the data?',
  'options': {'a': '20%', 'b': '40%', 'c': '60%', 'd': '80%'},
  'correct': 'd'},
 '3': {'no': '3',
  'mcq': 'What is the name of the AI platform used in the project work assignment?',
  'options': {'a': 'Tableau',
   'b': 'ChatGPT',
   'c': 'Both A and B',
   'd': 'Neither A nor B'},
  'correct': 'c'},
 '4': {'no': '4',
  'mcq': 'What is the purpose of the case studies in the analytics curriculum?',
  'options': {'a': 'To teach students how to clean data',
   'b': 'To teach students how to analyze data',
   'c': 'To show the impact and potential of technology',
   'd': 'To teach students how to visualize data'},
  'correct

In [133]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [134]:
quiz_table_data

[{'MCQ': 'What is the primary challenge students face when working on project assignments?',
  'Choices': 'a: Cleaning the data | b: Analyzing the data | c: Visualizing the data | d: Interpreting the data',
  'Correct': 'a'},
 {'MCQ': 'What percentage of the analysis process is devoted to prepping the data?',
  'Choices': 'a: 20% | b: 40% | c: 60% | d: 80%',
  'Correct': 'd'},
 {'MCQ': 'What is the name of the AI platform used in the project work assignment?',
  'Choices': 'a: Tableau | b: ChatGPT | c: Both A and B | d: Neither A nor B',
  'Correct': 'c'},
 {'MCQ': 'What is the purpose of the case studies in the analytics curriculum?',
  'Choices': 'a: To teach students how to clean data | b: To teach students how to analyze data | c: To show the impact and potential of technology | d: To teach students how to visualize data',
  'Correct': 'c'},
 {'MCQ': 'What type of data is used in the project work assignment?',
  'Choices': 'a: Only numerical data | b: Only categorical data | c: Bot

In [135]:
df=pd.DataFrame(quiz_table_data)

In [136]:
df

,MCQ,Choices,Correct
0,What is the primary challenge students face wh...,a: Cleaning the data | b: Analyzing the data |...,a
1,What percentage of the analysis process is dev...,a: 20% | b: 40% | c: 60% | d: 80%,d
2,What is the name of the AI platform used in th...,a: Tableau | b: ChatGPT | c: Both A and B | d:...,c
3,What is the purpose of the case studies in the...,a: To teach students how to clean data | b: To...,c
4,What type of data is used in the project work ...,a: Only numerical data | b: Only categorical d...,c


In [ ]:
df.to_csv("gen.csv",index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'11_09_2024_13_14_57'